In [ ]:
from ast import Param

from pyomo.environ import *
from pyomo.opt import SolverStatus, TerminationCondition
from pyomo.core import ConcreteModel
from pyomo.common.config import PositiveInt

import os
import math
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import networkx as nx
import plotly.express as px
import plotly.graph_objects as go

In [ ]:
def plot_tsp_tour(coords, tour, title=None, show_positions=True):
    """
    coords: dict {node: (x, y)}
    tour: список вершин у порядку обходу, наприклад [1, 23, 38, ..., 1]
          якщо останній елемент не дорівнює першому, ми автоматично замкнемо тур
    show_positions: якщо True — підписуємо вершини як "node (step)"
    """

    if tour[0] != tour[-1]:
        tour = list(tour) + [tour[0]]

    start = tour[0]

    position_in_tour = {
        node: idx for idx, node in enumerate(tour[:-1])
    }

    if not isinstance(coords, dict):
        coords = {i: (coords[i][0], coords[i][1]) for i in range(len(coords))}

    plt.figure(figsize=(10, 10))

    for k in range(len(tour) - 1):
        i = tour[k]
        j = tour[k+1]
        x_i, y_i = coords[i]
        x_j, y_j = coords[j]

        dx = x_j - x_i
        dy = y_j - y_i

        plt.arrow(x_i, y_i, dx, dy, length_includes_head=True, head_width=1, head_length=1.75, linewidth=1.0, alpha=0.8, zorder=1)

        # plt.plot([x_i, x_j], [y_i, y_j], color="gray", linewidth=1, zorder=1)

    xs = [coords[i][0] for i in coords]
    ys = [coords[i][1] for i in coords]
    plt.scatter(xs, ys, s=30, color="blue", zorder=2, label="Other Nodes")

    x_s, y_s = coords[start]
    plt.scatter([x_s], [y_s], s=120, color="red", edgecolors="black", zorder=3, label="Start Node")

    for node, (x, y) in coords.items():
        if show_positions and node in position_in_tour:
            step = position_in_tour[node]
            text = f"{node} ({step})"
        else:
            text = str(node)
        plt.text(x, y, text, fontsize=10, ha="right", va="bottom")

    plt.title(title if title else "TSP Tour")
    plt.xlabel("X")
    plt.ylabel("Y")
    plt.gca().set_aspect("equal", adjustable="box")
    plt.grid(True, linestyle="--", alpha=0.3)
    plt.legend(loc="best")
    plt.tight_layout()
    plt.show()

In [ ]:
n_value = 100
s_value = 1
k_value = 20

xcoord_list = [
    86, 19, 8, 40, 36, 33, 22, 18, 91, 74,
    16, 80, 59, 9, 8, 16, 32, 34, 69, 82,
    8, 76, 30, 96, 88, 94, 74, 58, 33, 62,
    80, 40, 5, 25, 94, 59, 48, 40, 24, 32,
    48, 18, 16, 53, 17, 50, 49, 82, 38, 10,
    98, 63, 73, 20, 53, 15, 75, 42, 85, 84,
    51, 78, 29, 95, 13, 10, 89, 34, 42, 15,
    34, 17, 53, 40, 63, 86, 51, 25, 52, 50,
    31, 90, 39, 94, 92, 87, 14, 82, 86, 26,
    73, 98, 36, 25, 64, 53, 39, 86, 93, 76,
]

ycoord_list = [
    29, 88, 42, 99, 100, 8, 30, 5, 41, 52,
    35, 9, 28, 73, 92, 41, 28, 84, 64, 51,
    83, 59, 19, 34, 18, 32, 96, 72, 69, 34,
    96, 75, 55, 75, 52, 47, 29, 18, 66, 64,
    12, 97, 7, 15, 20, 81, 21, 88, 55, 77,
    9, 50, 49, 77, 60, 68, 33, 71, 2, 88,
    93, 15, 88, 69, 97, 35, 99, 83, 44, 15,
    38, 56, 21, 59, 1, 93, 93, 34, 65, 98,
    23, 65, 14, 81, 39, 82, 65, 78, 26, 20,
    48, 98, 21, 70, 100, 68, 1, 77, 42, 63,
]

xcoord_data = {i + 1: xcoord_list[i] for i in range(n_value)}
ycoord_data = {i + 1: ycoord_list[i] for i in range(n_value)}


In [ ]:
class KCycleTSPModel:
    """
    Проста обгортка над гібридною k-cycle моделлю:
    - всередині будує Pyomo-модель
    - розв'язує 3 варіанти: flow, MTZ, hybrid через NEOS
    """

    def __init__(self, xcoord_list, ycoord_list,
                 s=1, k=None,
                 neos_email=None,
                 optimizer="cplex"):
        # --- зберігаємо вихідні дані ---
        assert len(xcoord_list) == len(ycoord_list), "xcoord_list і ycoord_list мають бути однакової довжини"

        self.n = len(xcoord_list)
        self.s = s
        self.k = k if k is not None else self.n - 1
        self.optimizer = optimizer

        # NEOS e-mail
        if neos_email is not None:
            os.environ["NEOS_EMAIL"] = neos_email

        # NEOS solver manager
        self.neos = SolverManagerFactory("neos")

        # координати як словники {i: value}, 1-based індексація
        self.xcoord_data = {i + 1: float(xcoord_list[i]) for i in range(self.n)}
        self.ycoord_data = {i + 1: float(ycoord_list[i]) for i in range(self.n)}

        # --- рахуємо d[i,j] як в AMPL: round(sqrt(...)) ---
        self.d_data = {}
        for i in range(1, self.n + 1):
            for j in range(1, self.n + 1):
                if i != j:
                    dx = self.xcoord_data[i] - self.xcoord_data[j]
                    dy = self.ycoord_data[i] - self.ycoord_data[j]
                    dist = math.sqrt(dx * dx + dy * dy)
                    self.d_data[(i, j)] = int(round(dist))

        # будуємо Pyomo-модель
        self._create_tsp_model()

    def _create_tsp_model(self):
        """
        Створює Pyomo-модель k-циклу для задачі TSP (flow + MTZ, гібридна модель).
        Повертає об'єкт model.
        """
        model = ConcreteModel()

        # --- Model Parameters ---
        model.n = Param(initialize=self.n, within=PositiveIntegers)
        model.s = Param(initialize=self.s, within=PositiveIntegers)
        model.k = Param(initialize=self.k, within=PositiveIntegers)

        # --- Nodes & Arcs ---
        model.NODES = RangeSet(1, model.n)

        def arcs_init(model_):
            return [(i, j) for i in model_.NODES for j in model_.NODES if i != j]

        model.ARCS = Set(dimen=2, initialize=arcs_init)

        # --- x_coord, y_coord, d ---
        model.x_coord = Param(model.NODES, initialize=self.xcoord_data)
        model.y_coord = Param(model.NODES, initialize=self.ycoord_data)

        model.d = Param(model.ARCS, within=NonNegativeReals, initialize=self.d_data)

        # --- x, y, z, u variables ---
        model.x = Var(model.ARCS, domain=Binary)
        model.y = Var(model.NODES, domain=Binary)
        model.z = Var(model.ARCS, domain=NonNegativeReals)

        def u_bounds(m_, i):
            return (1, m_.k)

        model.u = Var(model.NODES, domain=Integers, bounds=u_bounds)

        # --- Objective Function ---
        def obj_rule(m_):
            return sum(m_.d[i, j] * m_.x[i, j] for (i, j) in m_.ARCS)

        model.dk_min = Objective(rule=obj_rule, sense=minimize)

        # --- Constraints con2, con3, con4 ---
        def con2_rule(m_, i):
            if i == m_.s:
                rhs = 1
            else:
                rhs = m_.y[i]
            return sum(m_.x[i, j] for j in m_.NODES if (i, j) in m_.ARCS) == rhs

        model.con2 = Constraint(model.NODES, rule=con2_rule)

        def con3_rule(m_, i):
            if i == m_.s:
                rhs = 1
            else:
                rhs = m_.y[i]
            return sum(m_.x[j, i] for j in m_.NODES if (j, i) in m_.ARCS) == rhs

        model.con3 = Constraint(model.NODES, rule=con3_rule)

        def con4_rule(m_):
            return sum(m_.y[i] for i in m_.NODES if i != m_.s) == m_.k

        model.con4 = Constraint(rule=con4_rule)

        # --- Flow-based Part (con5 - con7) ---
        def con5_rule(m_, i, j):
            return m_.z[i, j] - m_.k * m_.x[i, j] <= 0

        model.con5 = Constraint(model.ARCS, rule=con5_rule)

        def con6_1_rule(m_):
            s = value(m_.s)
            return sum(m_.z[s, j] for j in m_.NODES if (s, j) in m_.ARCS) == m_.k

        model.con6_1 = Constraint(rule=con6_1_rule)

        def con6_2_rule(m_):
            s = value(m_.s)
            return sum(m_.z[j, s] for j in m_.NODES if (j, s) in m_.ARCS) == 0

        model.con6_2 = Constraint(rule=con6_2_rule)

        def con7_rule(m_, i):
            if i == m_.s:
                return Constraint.Skip
            outgoing = sum(m_.z[i, j] for j in m_.NODES if (i, j) in m_.ARCS)
            incoming = sum(m_.z[j, i] for j in m_.NODES if (j, i) in m_.ARCS)
            return outgoing - incoming == -m_.y[i]

        model.con7 = Constraint(model.NODES, rule=con7_rule)

        # --- MTZ Part (con15) ---
        def con15_rule(m_, i, j):
            if i == m_.s or j == m_.s:
                return Constraint.Skip
            return m_.u[i] - m_.u[j] + m_.k * m_.x[i, j] <= m_.k - 1

        model.con15 = Constraint(model.ARCS, rule=con15_rule)

        self.model = model

    def _print_solution(self, label, show_u=False, x_threshold=0.5):
        model = self.model
        print(f"dk_min ({label}) =", value(model.dk_min))

        print(f"Display x ({label}):")
        for i in model.NODES:
            for j in model.NODES:
                if (i, j) in model.ARCS:
                    x_val = value(model.x[i, j], exception=False)
                    if x_val is not None and x_val >= x_threshold:
                        print(f"{i} -> {j} [{x_val:.2f}]")

        if show_u:
            print(f"Display u ({label}):")
            s_val = value(model.s)
            for i in model.NODES:
                if i == s_val:
                    continue

                y_val = value(model.y[i], exception=False)
                if y_val is None or y_val <= 0.5:
                    continue

                u_val = value(model.u[i], exception=False)
                print(f"{i}\t{u_val}")


    def get_tour(self, start=None, threshold=0.5, max_steps=None):
        """
        Відновлює тур з поточного розв'язку:
        - start: з якої вершини починати (за замовчуванням s)
        - threshold: поріг для x[i,j], щоб вважати дугу обраною
        - max_steps: захист від зациклення (якщо None, беремо n+1)

        Повертає список вершин у порядку обходу, наприклад [1, 23, 38, ..., 1].
        """
        model = self.model
        if start is None:
            start = int(value(model.s))

        succ = {}
        for i in model.NODES:
            for j in model.NODES:
                if (i, j) in model.ARCS:
                    x_val = value(model.x[i, j])
                    if x_val is not None and x_val >= threshold:
                        succ[i] = j

        tour = [start]
        current = start
        visited = {start}

        if max_steps is None:
            max_steps = len(list(model.NODES)) + 1

        for _ in range(max_steps):
            if current not in succ:
                break
            nxt = succ[current]
            tour.append(nxt)
            if nxt == start:
                break
            if nxt in visited:
                break
            visited.add(nxt)
            current = nxt

        return tour

    def get_coords_dict(self):
        """
        Повертає coords у форматі {node: (x, y)},
        сумісному з plot_tsp_tour.
        """
        return {
            i: (self.xcoord_data[i], self.ycoord_data[i])
            for i in self.xcoord_data
        }


    def solve_flow_based_problem(self, tee=False):
        """
        problem1: flow-based формулювання (x, y, z, con2, con3, con4, con5, con6_1, con6_2, con7)
        """

        model = self.model
        print("=== Problem 1 (flow-based) ===")

        # flow-constraints
        model.con5.activate()
        model.con6_1.activate()
        model.con6_2.activate()
        model.con7.activate()

        # MTZ-constraints
        model.con15.deactivate()

        results = self.neos.solve(model, opt=self.optimizer, tee=tee)

        if (results.solver.status == SolverStatus.ok and
            results.solver.termination_condition in [TerminationCondition.optimal,
                                                     TerminationCondition.feasible]):

            t = getattr(results.solver, "time", None)
            if t is not None:
                print("Solve time (flow-based):", t)

            self._print_solution("flow-based", show_u=False)

        else:
            print("NEOS failed for flow-based problem")
            print("  status           =", results.solver.status)
            print("  termination_cond =", results.solver.termination_condition)

        return results

    def solve_MTZ_problem(self, tee=False):
        """
        problem2: MTZ-формулювання (x, y, u, con2, con3, con4, con15)
        """

        model = self.model
        print("=== Problem 2 (MTZ) ===")

        # flow-constraints
        model.con5.deactivate()
        model.con6_1.deactivate()
        model.con6_2.deactivate()
        model.con7.deactivate()

        # MTZ-constraints
        model.con15.activate()

        results = self.neos.solve(model, opt=self.optimizer, tee=tee)

        if (results.solver.status == SolverStatus.ok and
            results.solver.termination_condition in [TerminationCondition.optimal,
                                                     TerminationCondition.feasible]):

            t = getattr(results.solver, "time", None)
            if t is not None:
                print("Solve time (MTZ):", t)

            self._print_solution("MTZ", show_u=True)

        else:
            print("NEOS failed for problem2")
            print("  status           =", results.solver.status)
            print("  termination_cond =", results.solver.termination_condition)

        return results

    def solve_hybrid_problem(self, tee=False):
        """
        problem3: гібридна модель (flow + MTZ)
        """

        model = self.model
        print("=== Problem 3 (hybrid) ===")

        # Activate All
        model.con2.activate()
        model.con3.activate()
        model.con4.activate()
        model.con5.activate()
        model.con6_1.activate()
        model.con6_2.activate()
        model.con7.activate()
        model.con15.activate()

        results = self.neos.solve(model, opt=self.optimizer, tee=tee)

        if (results.solver.status == SolverStatus.ok and
            results.solver.termination_condition in [TerminationCondition.optimal,
                                                     TerminationCondition.feasible]):

            t = getattr(results.solver, "time", None)
            if t is not None:
                print("Solve time (Hybrid):", t)

            self._print_solution("hybrid", show_u=True)

        else:
            print("NEOS failed for Hybrid")
            print("  status           =", results.solver.status)
            print("  termination_cond =", results.solver.termination_condition)

        return results

In [ ]:
solver = KCycleTSPModel(
    xcoord_list=xcoord_list,
    ycoord_list=ycoord_list,
    s=s_value,
    k=k_value,
    neos_email="antonsivko480@gmail.com",
    optimizer="cplex"
)

In [ ]:
solver.solve_flow_based_problem()

tour = solver.get_tour()
coords = solver.get_coords_dict()

plot_tsp_tour(coords, tour)